Run dependencies

In [ ]:
!pip install -q numpy
!pip install h5py
!pip install wget

In [5]:
import numpy as np
from matplotlib import pyplot as plt

import time
import wget
import importlib
import os.path
import h5py
from tqdm.auto import tqdm

from sklearn import datasets
from sklearn import preprocessing
import scipy.sparse

from index_class import ivf_index
from index_class import cluster_index
from index_class import data_io
import results

import test 

np.random.seed(101) #Fix the seed for reproducible results.

### Loading Data

Set `dataset` to the name of the dataset to index -- `'glove-100'` for Glove-100 and `'lastfm-64'` for Last.fm

In [6]:
dataset = 'glove-100' #set to 'lastfm-64' for Last.fm experiments.

Download the dataset

In [ ]:
# Download the dataset
wget.download('http://ann-benchmarks.com/glove-100-angular.hdf5') #glove-100
wget.download('http://ann-benchmarks.com/lastfm-64-dot.hdf5') #lastfm-64

Set parameters of index for each dataset

In [7]:
glove_dict = {'f'        : "glove-100-angular.hdf5", 
            'n_clusters' : 1024,
            'M_PQ'       : 25,
            'Scann_t'    : 0.2/8,
            'n_probe'    : 128}

lastfm_dict = {'f'       : "lastfm-64-dot.hdf5", 
            'n_clusters' : 256,
            'M_PQ'       : 16,
            'Scann_t'    : 0.2/8,
            'n_probe'    : 32}

In [8]:
data_dict = glove_dict if dataset == 'glove-100' else lastfm_dict

Load the dataset, normalize for dor-product search and sample `num_queries` queries from test set for testing.

In [ ]:
f = h5py.File(data_dict['f'], 'r')
#Sample desired number of data points and query points.
n, num_queries = f['train'].shape[0], 1000

X = preprocessing.normalize(f['train'][:]) #normalized for cosine similarity.

iq = np.random.choice(f['test'].shape[0], size=num_queries, replace=False)
Q = f['test'][np.sort(iq)] # Sample num_queries from the test queries.

# List and build indexes

In this section we index `X` the dataset using each kind of method -- `k-means`, `PCPQ`, `Q-PCPQ`, `ScaNN`, `APCPQ` and `Q-APCPQ`

In [9]:
#Setting global variables for index parameters.
n_clusters = data_dict['n_clusters'] #Number of clusters in initial clustering of data
M_PQ = data_dict['M_PQ'] #m
K_PQ16, K_PQ256 = 16, 256 #k
SQ = 8 #Scalar quantizer, s

Initial clustering of datapoints `X` into `n_clusters` partitions. The data is clustered once, saved and used for each indexing method.

In [5]:
ivfix = ivf_index.IVFIndex(num_cluster_indexes=n_clusters, data_name=data_name)
ivfix.build_ivf_index(X, save_index=True)

IVF Indexing loaded from file: results/IVFIndex_glove-100_1024


List of the methods of indexing is stored in `cluster_index_types`

In [11]:
cluster_index_types = [
                        (cluster_index.PQIndex, [K_PQ16, M_PQ, cluster_index.KMeansIndex(K_PQ16)], {}, 'k-means++'),
                       
                       (cluster_index.PQIndex, [K_PQ16, M_PQ, cluster_index.ProjCIndex(K_PQ16)],
                        {'initializer':cluster_index.ProjCIndex.initialize_kmeans, 'descent':True}, 'PCPQ'),
                       
                       (cluster_index.PQIndex, [K_PQ16, M_PQ, cluster_index.ProjCIndex(K_PQ16)], 
                        {'initializer':cluster_index.ProjCIndex.initialize_kmeans, 'descent':True, 'SQ':SQ}, 'Q-PCPQ'),                    
                       
                       (cluster_index.PQIndex, [K_PQ16, M_PQ, cluster_index.ScannIndex(K_PQ16, T=data_dict['Scann_t'])],
                        {'descent':True}, 'ScaNN'),
                       
                       (cluster_index.PQIndex, [K_PQ16, M_PQ, cluster_index.ScannIndex(K_PQ16, T=data_dict['Scann_t'])],
                        {'descent':True, 'project':True}, 'APCPQ'),
                       
                       (cluster_index.PQIndex, [K_PQ16, M_PQ, cluster_index.ScannIndex(K_PQ16, T=data_dict['Scann_t'])],
                        {'descent':True, 'project':True, 'SQ':SQ}, 'Q-APCPQ'),
                       
                        (cluster_index.PQIndex, [K_PQ256, M_PQ, cluster_index.KMeansIndex(K_PQ256)], {}, 'k-means++'),
                       
                       (cluster_index.PQIndex, [K_PQ256, M_PQ, cluster_index.ProjCIndex(K_PQ256)], 
                       {'initializer':cluster_index.ProjCIndex.initialize_kmeans, 'descent':True}, 'PCPQ'),
                       
                       (cluster_index.PQIndex, [K_PQ256, M_PQ, cluster_index.ProjCIndex(K_PQ256)], 
                       {'initializer':cluster_index.ProjCIndex.initialize_kmeans, 'descent':True, 'SQ':SQ}, 'Q-PCPQ'),                       
                       
                       (cluster_index.PQIndex, [K_PQ256, M_PQ, cluster_index.ScannIndex(K_PQ256, T=data_dict['Scann_t'])],
                        {'descent':True}, 'ScaNN'),
                       
                       (cluster_index.PQIndex, [K_PQ256, M_PQ, cluster_index.ScannIndex(K_PQ256, T=data_dict['Scann_t'])],
                        {'descent':True, 'project':True}, 'APCPQ'),
                       
                       (cluster_index.PQIndex, [K_PQ256, M_PQ, cluster_index.ScannIndex(K_PQ256, T=data_dict['Scann_t'])],
                        {'descent':True, 'project':True, 'SQ':SQ}, 'Q-APCPQ')
                      ]


Each method listed in `cluster_index_types` is used to index the data. Each index is saved.

In [ ]:
#Build the indexes if not already saved otherwise load
for (ix_class, initargs, buildkwargs, label) in cluster_index_types: 
    cix = ivfix.build_cluster_indexes(ix_class, *initargs, save_index=True, **buildkwargs)

# Recall of top 1@N.

Plotting the recall for `1@N` for different values of `N`, i.e. results of Figure 4 in paper.

In [12]:
N_max = 30 #Max value of N desired for plot
N_list = np.arange(1, N_max + 1, 2)
n_probe = data_dict['n_probe'] #Number of first level partitioning probed 

top_k = 1
eps_scaling = 1.001

Compute exact scores

In [ ]:
N_max = max(N_list)
exact_scores = np.zeros((Q.shape[0], N_max))
for i in tqdm(range(Q.shape[0])): exact_scores[i] = np.sort(np.partition(X@(-Q[i]), N_max-1)[:N_max])

Compute average recall1@N for each method over queries `Q`

In [ ]:
rs_N = []
for i in tqdm(range(0, len(cluster_index_types))):
    r = []
    (ix_class, initargs, buildkwargs, label) = cluster_index_types[i] 
    cix = ivfix.build_cluster_indexes(ix_class, *initargs, save_index=True, **buildkwargs)
    for i_n in tqdm(range(len(N_list)), leave=False):
        N_val = N_list[i_n]
        r.append(np.mean(test.test_score_recall(X,Q,exact_scores,top_k,N_val,ivfix,eps_scaling,*[n_probe, N_val, N_val, cix])))
    rs_N.append(r)

In [42]:
label = "%s_1@N_list"%dataset #Label to save results
R = results.Results(cluster_index_types, rs_N, **{'N_list':N_list, 'top_k':top_k, 'n_probe':n_probe, 'eps_scaling':eps_scaling})
R.save_results(label)
# R = results.Results.load_results(label) #To load results

Plot figures from Figure 4 in paper.

In [ ]:
index_split = [[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 10, 11]]
fig, axs = plt.subplots(2, 2)
fig.set_size_inches(10, 8)
for i in range(len(index_split)):
    for i_n in index_split[i]: 
        axs[i//2][i%2].plot(R.kwargs['N_list'], R.results[i_n], label=R.index_types[i_n][3])
    axs[i//2][i%2].set_xlabel('N')
    axs[i//2][i%2].legend()
    axs[i//2][i%2].set_title('Recall1@N for %s, (%d - bit)' % (dataset, K_PQ/8))